In [1]:
#Dependecies
from config import api_key
import requests
import json
import pandas as pd
from census import Census
from us import states


In [2]:
# load the zipcode into a Dataframe
zipcode_list = pd.read_csv("..\CSVs\Armann\zipdemos.csv")
#zipcode_list['Zipcode'] = zipcode_list['Zipcode'].astype('object')
print(zipcode_list.head())
zipcode_list.info()

   zip_code       lat        lng         city  population  density  \
0     90001  33.97365 -118.24904  Los Angeles       58245   6856.0   
1     90002  33.94906 -118.24679  Los Angeles       54384   7016.7   
2     90003  33.96409 -118.27368  Los Angeles       75190   8007.9   
3     90004  34.07624 -118.31101  Los Angeles       59621   7545.6   
4     90005  34.05939 -118.30909  Los Angeles       36910  12078.9   

        county  
0  Los Angeles  
1  Los Angeles  
2  Los Angeles  
3  Los Angeles  
4  Los Angeles  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1802 entries, 0 to 1801
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   zip_code    1802 non-null   int64  
 1   lat         1802 non-null   float64
 2   lng         1802 non-null   float64
 3   city        1802 non-null   object 
 4   population  1802 non-null   int64  
 5   density     1802 non-null   float64
 6   county      1802 non-null   object 
dt

In [3]:
zipcode_list = zipcode_list.rename(columns={
    "zip_code": "Zipcode",
    "lat": "Latitude",
    "lng": "Longitude",
    "city": "City",
    "population": "Population",
    "density": "Density",
    "county": "County"
})

In [4]:
# Define Census API client
# can only obtain data upto 2020
c = Census(api_key, year = 2020)

# Define FIPS code for California
ca_fips = states.CA.fips
zip_code = "*" # or use "Census.ALL"

# Retrieve data for all zip codes in California
census_data = c.acs5.state_zipcode(('NAME','B19301_001E',
                                     'B19013_001E',
                                     'B17001_002E',
                                     'B01002_001E',
                                     'B25077_001E',
                                     'B25088_002E',
                                     'B01003_001E',
                                     'B02001_002E',
                                     'B02001_003E',
                                     'B02001_004E',
                                     'B02001_005E',
                                     'B03001_003E',
                                     'B02001_008E',

                                    'B08301_010E'), state_fips=ca_fips, zcta = zip_code)
# convert to dataframe
census_df= pd.DataFrame(census_data)

# column renaming
census_df = census_df.rename(
    columns = {
        'NAME': 'Name',
        'zip code tabulation area':'Zipcode',
        'B19301_001E': 'Income_per_capita',
        'B19013_001E': 'Median_household_income',
        'B17001_002E': 'Poverty',
        'B01002_001E': 'Median_age',
        'B25077_001E': 'Median_home_value',
        'B25088_002E': 'Median_monthly_owner_cost',
        'B01003_001E': 'Total_Population',
        'B02001_002E': 'White_Population',
        'B02001_003E': 'Black_Population',
        'B02001_004E': 'American_Indian_Population',
        'B02001_005E': 'Asian_Population',
        'B03001_003E': 'Hispanic_Population',
        'B02001_008E': 'Population_identifying_as_2+_races',
        'B08301_010E': 'Transit_public_transport'
        
    })
census_df = census_df.drop(columns=['Name'])
#configure the final Dataframe
census_df = census_df[[
    'Zipcode',
'Income_per_capita',
'Median_household_income',
'Poverty',
'Median_age',
'Median_home_value',
'Median_monthly_owner_cost',
'Total_Population',
'White_Population',
'Black_Population',
'American_Indian_Population',
'Asian_Population',
'Hispanic_Population',
'Population_identifying_as_2+_races',
'Transit_public_transport'
]]

census_df.head()

,Zipcode,Income_per_capita,Median_household_income,Poverty,Median_age,Median_home_value,Median_monthly_owner_cost,Total_Population,White_Population,Black_Population,American_Indian_Population,Asian_Population,Hispanic_Population,Population_identifying_as_2+_races,Transit_public_transport
0,29590,13288.0,30985.0,869.0,39.1,68600.0,1271.0,3543.0,718.0,2617.0,27.0,0.0,346.0,68.0,14.0
1,93306,23580.0,54450.0,15853.0,31.3,211100.0,1541.0,74296.0,51869.0,1727.0,1208.0,2455.0,48116.0,5176.0,228.0
2,93660,12475.0,39625.0,1057.0,26.7,171700.0,1114.0,4082.0,2231.0,40.0,0.0,0.0,3925.0,142.0,0.0
3,93110,53375.0,93264.0,1332.0,46.9,844300.0,3063.0,15777.0,12042.0,311.0,287.0,674.0,4251.0,702.0,185.0
4,93212,10656.0,42983.0,3911.0,34.4,162100.0,1160.0,22596.0,13033.0,2708.0,333.0,177.0,15641.0,2104.0,21.0


In [5]:
census_df['Zipcode'] = pd.to_numeric(census_df['Zipcode'], errors='coerce')

census_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33120 entries, 0 to 33119
Data columns (total 15 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Zipcode                             33120 non-null  int64  
 1   Income_per_capita                   32790 non-null  float64
 2   Median_household_income             33120 non-null  float64
 3   Poverty                             33120 non-null  float64
 4   Median_age                          33120 non-null  float64
 5   Median_home_value                   33120 non-null  float64
 6   Median_monthly_owner_cost           33120 non-null  float64
 7   Total_Population                    33120 non-null  float64
 8   White_Population                    33120 non-null  float64
 9   Black_Population                    33120 non-null  float64
 10  American_Indian_Population          33120 non-null  float64
 11  Asian_Population                    33120

In [6]:
# Save the DataFrame as a CSV
# Note: To avoid any issues later, use encoding="utf-8"
census_df.to_csv("..\CSVs\Katejoe\census_data.csv", encoding="utf-8", index=False)
#Save the dataframe as a JSON
census_df.to_json('..\CSVs\Katejoe\census.json',orient='records')

In [7]:
demographics = pd.merge(census_df, zipcode_list, how='inner', on='Zipcode')
demographics = demographics.drop(columns=['Population', 'Density'])
demographics= demographics._get_numeric_data()
demographics[demographics < 0] = 0
demographics = demographics.dropna()
demographics.head()
demographics.to_csv("..\CSVs\Katejoe\demographics_data.csv", encoding="utf-8", index=False)

In [8]:
# Save and convert the dataframe to a JSON string with orient='records'
demographics.to_json('..\CSVs\Katejoe\dem.json',orient='records')


In [9]:
demographics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1735 entries, 0 to 1754
Data columns (total 17 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Zipcode                             1735 non-null   int64  
 1   Income_per_capita                   1735 non-null   float64
 2   Median_household_income             1735 non-null   float64
 3   Poverty                             1735 non-null   float64
 4   Median_age                          1735 non-null   float64
 5   Median_home_value                   1735 non-null   float64
 6   Median_monthly_owner_cost           1735 non-null   float64
 7   Total_Population                    1735 non-null   float64
 8   White_Population                    1735 non-null   float64
 9   Black_Population                    1735 non-null   float64
 10  American_Indian_Population          1735 non-null   float64
 11  Asian_Population                    1735 no